# Instructions:

**Requirements:**


---


Cleversys output files consisting of 48 30-minute blocks of behavior with all
behaviors selected for output.

---

Make sure to check the box for interleaved data so the columns are Mouse 1 time, then Mouse 1 duration, etc, not all time then all durations.

---

All mice used must have a value in their mouse number info spot at Row 9 cells B,C,D,E in your excel output

---

Date and Time must be in cell 20B in the format YYYY-MM-DD HH:MM:SS"

---

In [1]:
!mkdir -p /content/Jumping_CleverSys/Raw

In [3]:
!mkdir -p /content/Jumping_CleverSys/Processed

In [4]:
cd /content/Jumping_CleverSys/Raw

/content/Jumping_CleverSys/Raw


In [48]:
#@title Import your raw CleverSys output files
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving 2022_06_28 adult Esr1CRE after Tamoxifen 2.xlsx to 2022_06_28 adult Esr1CRE after Tamoxifen 2.xlsx
User uploaded file "2022_06_28 adult Esr1CRE after Tamoxifen 2.xlsx" with length 90422 bytes


In [10]:
#@title Define Functions
# Define all Functions

# save it
def saveit(dark, MasterFile0, summary):
    date = str(dark['Start Time'].values[0])[:10]
    savename = str("/content/Jumping_CleverSys/Processed") + "/" + " " + date + " Processed " + str(dt.today().strftime('%Y-%m-%d')) + ".xlsx"

# modify for excel writing
    dark[['Behavior', 'Bin No', 'Start Time', 'Sex', 'Genotype', 'Start']] = dark[[
        'Behavior', 'Bin No', 'Start Time', 'Sex', 'Genotype', 'Start']].astype(str)
    MasterFile0[['Behavior', 'Bin No', 'Start Time', 'Sex', 'Genotype', 'Start']] = MasterFile0[[
        'Behavior', 'Bin No', 'Start Time', 'Sex', 'Genotype', 'Start']].astype(str)

    with pd.ExcelWriter(savename) as writer:
        dark.to_excel(writer, sheet_name='Dark')
        MasterFile0.to_excel(writer, sheet_name='All Data')
        summary.to_excel(writer, sheet_name='Sum in Dark')
        
        button
        
    return(savename)


# remaining functions

def analyze_cleversys(xls, mouse1, mouse2, mouse3, mouse4, 
                      m1geno, m2geno, m3geno,m4geno,
                      m1sex, m2sex, m3sex, m4sex):
    # import info
    names_and_dates = pd.read_excel(xls, 'HCSInfo')
    data = pd.read_excel(xls, 'HCSGrpBinTimesDuration')

    # make the numbers numeric
    x = range(2, len(data.columns), 1)
    for n in x:
        data.iloc[:, n] = pd.to_numeric(data.iloc[:, n], errors='coerce')

    data2 = data.drop([0, 1, 2, 3, 4, 5], axis=0).fillna(method='ffill')

    mice = [mouse1, mouse2, mouse3, mouse4]
    mice = [i for i in mice if i != 0]
    num_mice = len(mice)


# if there are 4 mice
    if num_mice == 4:
        data2.columns = [
            'Behavior', 'Bin No', 'Mouse 1 Times', 'Mouse 1 Duration',
            'Mouse 2 Times', 'Mouse 2 Duration', 'Mouse 3 Times',
            'Mouse 3 Duration', 'Mouse 4 Times', 'Mouse 4 Duration'
        ]

        data2.to_clipboard()

# if there are 3 mice
    if num_mice == 3:
        data2 = data2.iloc[:, :8]
        print(len(data2.columns))
        data2.columns = [
            'Behavior', 'Bin No', 'Mouse 1 Times', 'Mouse 1 Duration',
            'Mouse 2 Times', 'Mouse 2 Duration', 'Mouse 3 Times',
            'Mouse 3 Duration'
        ]


# if there are 2 mice
    if num_mice == 2:
        data2 = data2.iloc[:, :6]
        data2.columns = [
            'Behavior', 'Bin No', 'Mouse 1 Times', 'Mouse 1 Duration',
            'Mouse 2 Times', 'Mouse 2 Duration'
        ]


# if there is 1 mouse
    if num_mice == 1:
        data2 = data2.iloc[:, :4]
        data2.columns = [
            'Behavior', 'Bin No', 'Mouse 1 Times', 'Mouse 1 Duration'
        ]


# filter behaviors
    data3 = data2[data2['Behavior'].isin([
        'Groom', 'HVfromHC', 'HVfromRU', 'HangCudl', 'Jump', 'Pause', 'RearUp',
        'RemainHC', 'RemainHV', 'RemainLw', 'RemainPR', 'RemainRU', 'ReptJump',
        'Sleep', 'Stationa', 'Stretch', 'TravDist.', 'Turn'
    ])]


# get start times sorted out
    start0 = names_and_dates.iloc[18:19, 1:2]
    val = start0['Unnamed: 1'].values[0]

    dstart = datetime.datetime.strptime(val, '%Y-%m-%d %H:%M:%S')
    starts, starts1 = [], []
    starts.append(dstart)
    for i in range(0, 47):
        starts.append(starts[i] + timedelta(minutes=30))

    for time in range(0, (len(starts))):
        starts[time] = starts[time].strftime('%Y-%m-%d %H:%M:%S')

    all_start = starts * len(set(data3['Behavior']))

    data3['Start Time'] = all_start

# split into one dataframe per mouse

    Mdf = data3[[
        'Behavior', 'Bin No', 'Mouse 1 Times', 'Mouse 1 Duration', 'Start Time'
    ]].copy()

    Mdf['Mouse'] = mouse1
    Mdf['Sex'] = m1sex
    Mdf['Genotype'] = m1geno
    Mdf.rename(columns={'Mouse 1 Times': 'Times'}, inplace=True)
    Mdf.rename(columns={'Mouse 1 Duration': 'Duration'}, inplace=True)

    if num_mice >= 2:
        M2df = data3[[
            'Behavior', 'Bin No', 'Mouse 2 Times', 'Mouse 2 Duration', 'Start Time'
        ]].copy()

        M2df['Mouse'] = mouse2
        M2df['Sex'] = m2sex
        M2df['Genotype'] = m2geno
        M2df.rename(columns={'Mouse 2 Times': 'Times'}, inplace=True)
        M2df.rename(columns={'Mouse 2 Duration': 'Duration'}, inplace=True)

    if num_mice >= 3:
        M3df = data3[[
            'Behavior', 'Bin No', 'Mouse 3 Times', 'Mouse 3 Duration', 'Start Time'
        ]].copy()

        M3df['Mouse'] = mouse3
        M3df['Sex'] = m3sex
        M3df['Genotype'] = m3geno
        M3df.rename(columns={'Mouse 3 Times': 'Times'}, inplace=True)
        M3df.rename(columns={'Mouse 3 Duration': 'Duration'}, inplace=True)

    if num_mice == 4:
        M4df = data3[[
            'Behavior', 'Bin No', 'Mouse 4 Times', 'Mouse 4 Duration', 'Start Time'
        ]].copy()

        M4df['Mouse'] = mouse4
        M4df['Sex'] = m4sex
        M4df['Genotype'] = m4geno
        M4df.rename(columns={'Mouse 4 Times': 'Times'}, inplace=True)
        M4df.rename(columns={'Mouse 4 Duration': 'Duration'}, inplace=True)

# put it back together
    if num_mice == 2:
        Mdf = Mdf.append(M2df, ignore_index=True)

    if num_mice == 3:
        Mdf = Mdf.append(M2df, ignore_index=True).append(M3df,
                                                         ignore_index=True)
    if num_mice == 4:
        Mdf = Mdf.append(M2df, ignore_index=True).append(
            M3df, ignore_index=True).append(M4df, ignore_index=True)

    MasterFile0 = Mdf.copy()
# Set the dates up to be usable
    MasterFile0['Start Time'] = pd.to_datetime(MasterFile0['Start Time'],
                                               utc=True)
    MasterFile0['Start'] = [
        datetime.datetime.time(d) for d in MasterFile0['Start Time']
    ]

# put a fake day value in so I can compare across days
    fakedate = str(datetime.datetime.strptime('2000-01-01', '%Y-%m-%d').date())
    MasterFile0['Start'] = pd.to_datetime(fakedate + " " +
                                          MasterFile0['Start'].astype(str))

# Make the values the right types so we can use them
   # MasterFile0[["Times",
    #            "Duration"]] = MasterFile0[["Times",
    #                   "Duration"]].apply(pd.to_numeric)
    MasterFile0['Mouse'] = MasterFile0['Mouse'].astype(str)
    MasterFile0['Start'] = pd.to_datetime(MasterFile0['Start'], utc=True)

# extract times in the dark
    dark = MasterFile0[~((MasterFile0['Start'] > '2000-01-01 06:00:00') &
                         (MasterFile0['Start'] <= '2000-01-01 18:00:00'))]

# get summary data
    summary = dark.drop(columns=['Bin No', 'Start'])
    summary = dark.drop(columns=['Bin No', 'Start']).groupby([
        'Genotype',
        'Behavior',
        'Sex',
        'Mouse',
    ]).agg(['sum', 'mean', 'sem']).reset_index()
    summary.columns = [
        'Genotype', 'Behavior', 'Sex', 'Mouse', 'Sum_Times', 'Mean_Times',
        'SEM_Times', 'Sum_Duration', 'Mean_Duration', 'SEM_Duration'
    ]
    print("File Processed")
    return (dark, MasterFile0, summary)




def analyze(): 
    # Process files and save them

    # read each file and process it
    import warnings
    import datetime as datetime
    globber =  "/content/Jumping_CleverSys/Raw" + "/*.xlsx"
    globbed_files = glob.glob(globber)

    warnings.simplefilter(action='ignore', category=FutureWarning)

    for xl in globbed_files:
        print(xl)
        # get each file and parse out the mouse / geno / hemisphere
        names_and_dates = pd.read_excel(xl, 'HCSInfo', index_col=0)
        data = pd.read_excel(xl, 'HCSGrpBinTimesDuration')

        mouse_ids = list(names_and_dates.loc['Mouse ID'])
        pad(mouse_ids, 0, 4)
        mouse_ids = mouse_ids[:4]

        if "Sex" in names_and_dates.index:
            sexes = list(names_and_dates.loc['Sex'])
            pad(sexes, 0, 4)
            sexes = sexes[:4]

        if "Genotype" in names_and_dates.index:
            genos = list(names_and_dates.loc['Genotype'])
            pad(genos, 0, 4)
            genos = genos[:4]

        m1geno = genos[0]
        m2geno = genos[1]
        m3geno = genos[2]
        m4geno = genos[3]

        mouse1 = mouse_ids[0]
        mouse2 = mouse_ids[1]
        mouse3 = mouse_ids[2]
        mouse4 = mouse_ids[3]

        m1sex = sexes[0]
        m2sex = sexes[1]
        m3sex = sexes[2]
        m4sex = sexes[3]

        dark, MasterFile0, summary = analyze_cleversys(xl, 
                                                       mouse1, mouse2, mouse3, mouse4, 
                                                       m1geno, m2geno, m3geno, m4geno,
                                                       m1sex, m2sex, m3sex, m4sex)
        savename = saveit(dark, MasterFile0, summary)
        
        
def pad(l, content, width):
    l.extend([content] * (width - len(l)))
    return l




In [11]:
#@title Move your raw files into the folder content/Jumping CleverSys/Raw Data then press the button
# set up the App: Fill in info here, then run next cells

import pandas as pd
from datetime import timedelta
import datetime as datetime
from datetime import datetime as dt
from ipyfilechooser import FileChooser
import ipywidgets as widgets
from ipywidgets import AppLayout
import glob
pd.options.mode.chained_assignment = None



button = widgets.Button(description="Click to Analyze",button_style='success')

def on_button_clicked(b):
    analyze()

button.on_click(on_button_clicked)


button
# deploy GUI

Button(button_style='success', description='Click to Analyze', style=ButtonStyle())

In [14]:
cd /content/Jumping_CleverSys/Processed

/content/Jumping_CleverSys/Processed


In [18]:
#@title Pick up processed files and produce final file: Make sure they are in the Processed Data folder
Name_you_want_to_save_the_final_file_as = "" #@param {type:"string"}
# pick up processed files and produce final file
#  This needs to run only when ready

headera = widgets.Label(value= "Click to Aggregate Data ONLY AFTER processed files are created:")

from ipywidgets import HBox, Label, VBox

def final_analysis():
    
    globber2 = "/content/Jumping CleverSys/Processed" + "/*.xlsx"
    globbed_files2 = glob.glob(globber2)

    dataframe = pd.DataFrame(
        columns=['Genotype', 'Behavior', 'Sex', 'Mouse', 'Sum_Duration', 'Sum_Times'])

    for xl in globbed_files2:
        # get each file
        data = pd.read_excel(xl, 'Sum in Dark', index_col=0)
        data2 = data[['Genotype', 'Behavior', 'Sex',
                      'Mouse', 'Sum_Duration', 'Sum_Times']]
        dataframe = pd.concat([dataframe, data2])


    # format the final save file
    # empty frame to add to
    allmice = pd.DataFrame(columns=['Mouse', 'Groom', 'HVfromHC', 'HVfromRU', 'HangCudl', 'Jump', 'Pause',
                                    'RearUp', 'RemainHC', 'RemainHV', 'RemainLw', 'RemainPR', 'RemainRU',
                                    'ReptJump', 'Sleep', 'Stationa', 'Stretch', 'TravDist.', 'Turn'])


    for m in set(dataframe['Mouse']):
        mouseframe = dataframe[dataframe['Mouse'] == m]

       # remove travel for now
        travelframe = mouseframe[mouseframe['Behavior'] == "TravDist."
                                 ]
        travelframe['Sum_Duration'] = travelframe['Sum_Times']

        notravframe = mouseframe[mouseframe['Behavior'] != "TravDist"]
        allframe = pd.concat([travelframe, notravframe], sort=False)

        allmice0 = allframe.pivot_table(index=['Mouse', 'Genotype', 'Sex'], columns='Behavior',
                                        values='Sum_Duration', aggfunc='sum').reset_index()

        allmice = pd.concat([allmice0, allmice], sort=False)


    # save the file
    saveitt = "/content/Jumping CleverSys/Processed" + "/" + str(Name_you_want_to_save_the_final_file_as.value) + ".xlsx"
    allmice.to_excel(saveitt)
    print("File saved!")
    

button2 = widgets.Button(description="Final Analysis",
                         button_style='danger'
                        )

def on_button_clicked(b):
    final_analysis()

button2.on_click(on_button_clicked)

button2

Button(button_style='danger', description='Final Analysis', style=ButtonStyle())

NameError: ignored